# CS486 - Artificial Intelligence
## Lesson 9 - Genetic Algorithms

A genetic algorithm is another stochastic approach to iteratively improve a goal. We'll use the AIMA library to implement a string matching genetic algorithm. 

In [ ]:
from helpers import romania
from aima.search import *
from aima.notebook import psource

# we'll use pandas to nicely display and filter data
from pandas import DataFrame

## String Matching

We are going to design a genetic algorithm that starts from a population of random strings that, over many generations, will produce a string that matches some target. First, let's choose a target string to match:

In [ ]:
target = "Azure FTW!" # start with around 10 characters

Next, e'll use the AIMA **`init_population`** function to generate our initial population:

```python
init_population(pop_size, gene_pool, ind_size)
```

Population size is a tradeoff between computation and memory. For this problem, `100` works well. **`gene_pool`** contains all the possible values that can be used to generate a member of the population. Our gene pool will be the following range of ASCII characters:

In [ ]:
pop_size = 100
ind_size = len(target)
gene_pool = [chr(x) for x in range(32, 123)]

print("Gene pool:", ''.join(gene_pool))

Finally, the `ind_size` is the number of genes in each individual. 

In [ ]:
population = init_population(pop_size,gene_pool,ind_size)

# we'll use a DataFrame to generate a nice table
DataFrame.from_dict({'Individual': population})

## Fitness 

Next, we need to define a function that quantifies the fitness of individuals in the population. The more fit an individual, the more likely the will combine genes with other members of the population, i.e. "reproduce". 

The fitness function takes an individual - a word in our case - and returns a number. The larger the number returned by the fitness function, the more fit the individual. 

In [ ]:
def fitness(word):
    return 0

In [ ]:
def fitness(word):
    score = 0
    for i in range(len(word)):
        if word[i] == target[i]:
            score += 1
    return score

The table below shows us the fitness of individuals in the initial population. These are the individuals that will contribute the most to the next generation. 

In [ ]:
df = DataFrame.from_dict({
    'Individual': population, 
    'Fitness': [fitness(i) for i in population]
}).sort_values(by=['Fitness'], ascending=False)

df[df['Fitness'] != 0]

The **`select`** and **`recombine`** functions in the AIMA library select individuals from the population and recombine their genes.

## Mutation

It's unlikely that our population will converge on the goal strictly through reproduction. An important gene may not be represented inour population at all. New genes are introduced into each generation through *mutation*. Mutations are introduced by adding a random gene to randomly selected individuals in each generation.  

Here is the mutation function used by the AIMA library: 

In [ ]:
psource(mutate)

Longer strings take longer to converge, so we'll make mutation a function of string size:

In [ ]:
mutation_rate = 1 / ind_size

## Re-populating

The following code will use the fitness function to produce a successive generation. 

In [ ]:
next_gen = [mutate(recombine(*select(2, population, fitness)), gene_pool, mutation_rate)
              for _ in population))]

The next generation should be significantly more fit than the previous: 

In [ ]:
DataFrame.from_dict({
    'Individual': next_gen, 
    'Fitness': [fitness(i) for i in next_gen]
}).sort_values(by=['Fitness'], ascending=False)

Let's iterate and watch the algorithm progress:

In [ ]:
import time
from IPython.core.display import HTML
from IPython.display import clear_output, display

gen_num = 0
next_gen = population[:]
fittest = max(next_gen,key=fitness)

while (fitness(fittest) != ind_size):
    next_gen = [mutate(recombine(*select(2, next_gen, fitness)), gene_pool, mutation_rate)
        for i in range(len(population))]

    gen_num += 1
    fittest = max(next_gen,key=fitness)
    
    if gen_num % 10 == 0:
        display(HTML("<h1>{}: {}</h1>".format(gen_num, ''.join(fittest))))
        clear_output(wait=True)
        
display(HTML("<h1>{}: {}</h1>".format(gen_num, ''.join(fittest))))

We can do all of that in one step using AIMA's **`genetic_algorithm`**:

In [ ]:
%%time
genetic_algorithm(population,fitness,gene_pool,ind_size)

# Writing a Blokus Agent

How can you write an agent using genetic algorithms?